# I19 : Provide Optional Behavior wiht Keyword Arguments

In [1]:
def flow_rate(weight_diff, time_diff,
              period=1, units_per_kg=1):
    return ((weight_diff / units_per_kg) / time_diff) * period

In [2]:
weight_diff = 0.5
time_diff = 3

pounds_per_hour = flow_rate(weight_diff, time_diff,
                           period=3600, units_per_kg=2.2)

- use position or keyword args
- set defaults with keyword
- optional should be always be passed by keyword

# I20 : Use None and Docstrings to Specify Dynamic Default Args

- when use non-static type as a keyword argument's default

In [9]:
from datetime import datetime

def log(message, when=datetime.now()):
    print('%s: %s' % (when, message))

In [10]:
import time
log('Hi there')
time.sleep(0.1)
log('Hi there')

2018-01-19 12:22:18.826450: Hi there
2018-01-19 12:22:18.826450: Hi there


In [11]:
def log(message, when=None):
    """Log a message with a timestamp.
    
    Args:
        message: Message to print.
        when: datetime of when the message occurred.
            Defaults to the present time.
    """
    when = datetime.now() if when is None else when
    print('%s: %s' % (when, message))

In [12]:
log('Hi there')
time.sleep(0.1)
log('Hi there')

2018-01-19 12:22:20.451248: Hi there
2018-01-19 12:22:20.555246: Hi there


In [13]:
import json

def decode(data, default={}):
    try:
        return json.loads(data)
    except ValueError:
        return default

In [14]:
def decode(data, default=None):
    """Load JSON data from a string.
    
    Args:
        data: JSON data to decode.
        default = Value to return if decoding fails.
            Defaults to an empty dictionary.
    """
    if default is None:
        default = {}
    try:
        return json.loads(data)
    except ValueError:
        return default      

# I21 : Enforce Clarity with Keyword-Only Arguments

In [16]:
def safe_division(number, divisor, ignore_overflow, 
                 ignore_zero_division):
    try:
        return number / divisor
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise

In [17]:
# ignore* are keyword only

def safe_division(number, divisor, 
                  ignore_overflow=False, 
                  ignore_zero_division=False):
    try:
        return number / divisor
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise

# I22 : Prefer Helper Classes Over Bookkeeping with Dictionaries and Tuples

In [18]:
class SimpleGradebook(object):
    def __init__(self):
        self._grades = {}
        
    def add_student(self, name):
        self._grades[name] = []
        
    def report_grade(self, name, score):
        self._grades[name].append(score)
    
    def average_grade(self, name):
        grades = self._grades[name]
        return sum(grades) / len(grades)

In [20]:
book = SimpleGradebook()
book.add_student('Kaden Cho')
book.report_grade('Kaden Cho', 90)
print(book.average_grade('Kaden Cho'))

90.0


In [21]:
class BySubjectGradebook(object):
    def __init__(self):
        self._grades = {}
        
    def add_student(self, name):
        self._grades[name] = {}
        
    def report_grade(self, name, subject, grade):
        by_subject = self._grades[name]
        grade_list = by_subject.setdefault(subject, [])
        grade_list.append(grade)
        
    def average_grade(self, name):
        by_subject = self._grades[name]
        total, count = 0, 0
        for grades in by_subject.values():
            total += sum(grades)
            count += len(grades)
        return total / count

In [22]:
book = BySubjectGradebook()
book.add_student('Albert')
book.report_grade('Albert', 'Math', 75)
book.report_grade('Albert', 'Math', 65)
book.report_grade('Albert', 'Gym', 95)
book.report_grade('Albert', 'Gym', 85)

In [28]:
class WeightedGradebook(object):
    def __init__(self):
        self._grades = {}
        
    def add_student(self, name):
        self._grades[name] = {}
        
    def report_grade(self, name, subject, score, weight):
        by_subject = self._grades[name]
        grade_list = by_subject.setdefault(subject, [])
        grade_list.append((score, weight))
        
    def average_grade(self, name):
        by_subject = self._grades[name]
        score_sum, score_count = 0, 0
        for subject, scores in by_subject.items():
            subject_avg, total_weight = 0, 0
            total_score = 0
            for score, weight in scores:
                total_score += score                
                total_weight += weight
                subject_avg = total_score / total_weight
            score_sum += subject_avg
            score_count += 1
        return score_sum / score_count    

> namedtuple & class refactoring

In [25]:
import collections
Grade = collections.namedtuple('Grade', ('score', 'weight'))

In [26]:
class Subject(object):
    def __init__(self):
        self._grades = []
        
    def report_grade(self, score, weight):
        self._grades.append(Grade(score, weight))
        
    def average_grade(self):
        total, total_weight = 0, 0
        for grade in self._grades:
            total += grade.score * grade.weight
            total_weight += grade.weight
        return total / total_weight

In [27]:
class Student(object):
    def __init__(self):
        self._subjects = {}
        
    def subject(self, name):
        if name not in self._subjects:
            self._subjects[name] = Subject()
        return self._subjects[name]
    
    def average_grade(self):
        total, count = 0, 0
        for subject in self._subjects.values():
            total += subject.average_grade()
            count += 1
        return total / count

In [29]:
class Gradebook(object):
    def __init__(self):
        self_students = {}
        
    def student(self, name):
        if name not in self._students:
            self_students[name] = Student()
        return self._students[name]

# I23 : Accept Functions for Simple Interfaces Instead of Classes

- *hook*

In [2]:
names = ['Socrates', 'Archimedes', 'Plato', 'Aristotle']
names.sort(key=lambda x: len(x))
names

['Plato', 'Socrates', 'Aristotle', 'Archimedes']

In [3]:
def log_missing():
    print('Key added')
    return 0

In [5]:
from collections import defaultdict

current = {'green': 12, 'blue': 3}
increments = [
    ('red', 5),
    ('blue', 17),
    ('orange', 9),
]
result = defaultdict(log_missing, current)
print('Before:', dict(result))
for key, amount in increments:
    result[key] += amount
print('After:', dict(result))

Before: {'green': 12, 'blue': 3}
Key added
Key added
After: {'green': 12, 'blue': 20, 'red': 5, 'orange': 9}


In [14]:
def increment_with_report(current, increments):
    added_count = 0
    
    def missing():
        nonlocal added_count
        added_count += 1
        return 0
    
    result = defaultdict(missing, current)
    for key, amount in increments:
        result[key] += amount
        
    return result, added_count


result, count = increment_with_report(current, increments)
assert count == 2

In [13]:
# encapsulation of the state

class CountMissing(object):
    def __init__(self):
        self.added = 0
        
    def missing(self):
        self.added += 1
        return 0


counter = CountMissing()
result = defaultdict(counter.missing, current)
for key, amount in increments:
    result[key] += amount
assert counter.added == 2

In [15]:
class BetterCountMissing(object):
    def __init__(self):
        self.added = 0
        
    def __call__(self):
        self.added += 1
        return 0


counter = BetterCountMissing()
counter()
assert callable(counter)

# I24 : Use @classmethod polymorphism to construct objects generically

In [17]:
class GenericWorker(object):
    def __init__(self, input_data):
        self.input_data = input_data
        self.result = None
        
    def map(self):
        raise NotImplementedError
        
    def reduce(self, other):
        raise NotImplementedError
        
    @classmethod
    def create_workers(cls, input_class, config):
        workers = []
        for input_data in input_class.generate_inputs(config):
            workers.append(cls(input_data))
        return workers

In [18]:
class LineCountWorker(GenericWorker):
    def map(self):
        data = self.input_data.read()
        self.result = data.count('\n')
    
    def reduce(self, other):
        self.result += other.result

In [20]:
class GenericInputData(object):
    def read(self):
        raise NotImplementedError
        
    @classmethod
    def generate_inputs(cls, config):
        raise NotImplementedError

In [21]:
import os

class PathInputData(GenericInputData):
    def __init__(self, path):
        super().__init__()
        self.path = path
        
    def read(self):
        return open(self.path).read()
    
    @classmethod
    def generate_inputs(cls, config):
        data_dir = config['data_dir']
        for name in os.listdir(data_dir):
            yield cls(os.path.join(data_dir, name))

In [22]:
def execute(workers):
    threads = [Thread(target=w.map) for w in workers]
    for thread in threads: thread.start()
    for thread in threads: thread.join()
        
    first, rest = workers[0], workers[1:]
    for worker in rest:
        first.reduce(worker)
    return first.result

In [23]:
def mapreduce(worker_class, input_class, config):
    workers = worekr_class.create_workers(input_class, config)
    return execute(workers)

In [ ]:
from tempfile import Temporary Directory

with TemporaryDirectory() as tmpdir:
    write_test_files(tmpdir)
    config = {'data_dir': tmpdir}
    result = mapreduce(LineCountWorker, PathInputData, config)

> Polymorphism	is	a	way	for	multiple	classes	in	a	hierarchy	to	implement	their	own	unique versions	of	a	method.	This	allows	many	classes	to	fulfill	the	same	interface	or	abstract base	class	while	providing	different	functionality	

# I25 : Initialize Parent Classess with super

In [24]:
class MyBaseClass(object):
    def __init__(self, value):
        self.value = value
        

class MyChildClass(MyBaseClass):
    def __init__(self):
        MyBaseClass.__init__(self, 5)

In [27]:
class TimesTwo(object):
    def __init__(self):
        self.value *= 2
        
    
class PlusFive(object):
    def __init__(self):
        self.value += 5

In [28]:
class OneWay(MyBaseClass, TimesTwo, PlusFive):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        TimesTwo.__init__(self)
        PlusFive.__init__(self)

In [29]:
class AnotherWay(MyBaseClass, PlusFive, TimesTwo):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        TimesTwo.__init__(self)
        PlusFive.__init__(self)

In [30]:
class TimesFive(MyBaseClass):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        self.value *= 5
        
    
class PlusTwo(MyBaseClass):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        self.value += 2

In [31]:
class ThisWay(TimesFive, PlusTwo):
    def __init__(self, value):
        TimesFive.__init__(self, value)
        PlusTwo.__init__(self, value)

In [34]:
# MRO ( method Resolution Order)

class Explicit(MyBaseClass):
    def __init__(self, value):
        super(__class__, self).__init__(value * 2)
        

class Implicit(MyBaseClass):
    def __init__(self, value):
        super().__init__(value * 2)
        
    
assert Explicit(10).value == Implicit(10).value

# I26 : Use Multiple Inheritance Only for Mix-in Utility Classes

- If	you	find	yourself	desiring	the	convenience	and	encapsulation	that	comes	with	multiple inheritance,	consider	writing	a	mix-in	instead.	A	mix-in	is	a	small	class	that	only	defines	a set	of	additional	methods	that	a	class	should	provide.	Mix-in	classes	don’t	define	their own	instance	attributes	nor	require	their	__init__	constructor	to	be	called. 

In [36]:
class ToDictMixin(object):
    def to_dict(self):
        return self._traverse_dict(self.__dict__)
    
    def _traverse_dict(self, instance_dict):
        output = {}
        for key, value in instance_dict.items():
            output[key] = self._traverse(key, value)
        return output
    
    def _traverse(self, key, value):
        if isinstance(value, ToDictMixin):
            return value.to_dict()
        elif isinstance(value, dict):
            return self._traverse_dict(value)
        elif isinstance(value, list):
            return [self._traverse(key, i) for i in value]
        elif hasattr(value, '__dict__'):
            return self._traverse_dict(value.__dict__)
        else:
            return value

In [37]:
class BinaryTree(ToDictMixin):
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right

In [38]:
tree = BinaryTree(10, 
    left=BinaryTree(7, right=BinaryTree(9)),
    right=BinaryTree(13, left=BinaryTree(11)))
print(tree.to_dict())

{'value': 10, 'left': {'value': 7, 'left': None, 'right': {'value': 9, 'left': None, 'right': None}}, 'right': {'value': 13, 'left': {'value': 11, 'left': None, 'right': None}, 'right': None}}


In [40]:
class BinaryTreeWithParent(BinaryTree):
    def __init__(self, value, left=None,
                right=None, parent=None):
        super().__init__(value, left=left, right=right)
        self.parent = parent
        
    def _traverse(self, key, value):
        if (isinstance(value, BinaryTreeWithParent) and 
           key == 'parent'):
            return value.value
        else:
            return super()._traverse(key, value)